### Información extraida del PDF

In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./los7HabitosGenteAltamenteEfectiva.pdf" #Ruta al archivo PDF
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs)) # Imprimimos la cantidad de partes del PDF

206


# Version paga

In [2]:
import os

os.environ["OPENAI_API_KEY"] = '' #API Key de OPENAI (version paga)

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Ahora la VectorDB es persistente, se guardan los datos en esta carpeta

In [4]:
import chromadb

persistent_client = chromadb.PersistentClient(path='./vectordb')

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) #el text splitter nos va a permitir partir los documentos con el tamaño de tokens que queramos

splits = text_splitter.split_documents(docs) #partimos el pdf en mas partes

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(), persist_directory='./vectordb') #creamos la base de datos de vectores

retriever = vectorstore.as_retriever() #unimos todo y damos la posibilidad de hacer consultas

### Resultados

In [6]:
retriever.get_relevant_documents('afilar la sierra') #consulta que nos va a traer los documentos mas reelevantes teniendo en cuenta nuestra consulta

c:\Users\fcore\RAG basico\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='su jardín florido no reserva una parcela para la maleza.  \n \nCuando ya tenemos autoconciencia, debemos elegir los prop ósitos y principios que pondremos en práctica \nen la vida; de otro modo alguna otra cosa llenará el vacío, y perderemos nuestra autoconcien cia para ser como \nanimales rastreros, cuya finalidad primordial es la supervivencia y la reproducción. Las personas que viven en \nese nivel no están viviendo, están siendo vividas. Están reaccionando, no to man conciencia de los dones \nsingulares que dormitan sin desarro llarse en su interior.  \nY para desarrollarlos no hay atajos. Rige la ley de la cosecha: siempre recogem os lo que sembramos, ni \nmás ni menos. La ley de la justicia es inmutable, y cuanto más coherentes seamos con los prin cipios correctos, \nmejor será nuestro juicio sobre cómo funciona el mundo, y más precisos nuestros paradigmas, nuestros mapas', metadata={'page': 186, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
 Docu

### LLM

In [7]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

### Crear prompt

In [8]:
system_prompt = (
    "Tú eres un asistente para tareas de respuesta a preguntas."
    "Usa los siguientes fragmentos de contexto recuperado para responder "
    "la pregunta. Si no sabes la respuesta, di que no "
    "sabes. Usa un máximo de tres oraciones y mantén la "
    "respuesta concisa."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


chain = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, chain)

### Obtener nuestra respuesta

In [9]:
results = rag.invoke({"input": "cual es el septimo habito?"})

In [10]:
results

{'input': 'cual es el septimo habito?',
 'context': [Document(page_content='aserrando.  \nEl séptimo hábito consiste en tomar tiempo para afilar la sierra. Engloba a todos los otros hábitos del \nmodelo de los siete hábitos por que es el que los hace posibles.  \n \n \nLas cuatro dimensio nes de la renovación  \n \n \nEl séptimo hábito es CP personal. Significa preservar y realzar el mayor bien que usted posee: usted \nmismo. Significa renovar las cua tro dimensiones de su naturaleza: la física, la espiritual, la mental y la \nsocial/emocional.  \nAunque  con diferentes palabras, la mayoría de las filosofías de la vida tratan implícita o explícitame nte \nsobre estas cuatro dimensio nes. El filósofo Herb Shepherd describe la vida sanamente equilibra da en torno a \ncuatro valores: la perspectiva (espiritual), la  autonomía (mental), la conexión (social) y el tono (físico). El gurú \nGeorge Shee -han se refiere a cuatro roles: ser un buen animal (físico), un buen artesano (mental), un b

In [11]:
results['answer']

'El séptimo hábito es tomar tiempo para afilar la sierra, que implica renovar las cuatro dimensiones de su naturaleza: física, espiritual, mental y social/emocional.'

In [12]:
rag.invoke({"input": "cual es el primer habito"})

{'input': 'cual es el primer habito',
 'context': [Document(page_content='El tercer hábito es el fruto personal, la realización práctica del primero y el segundo.  \nEl primer hábito dice: «Tú eres el creador. Tú estás a cargo de todo». Se basa en los cuatro privilegios \nhumanos de la imaginació n, la conciencia moral, la voluntad independiente y, en particular, la autoconciencia. \nNos permite decir: «Éste es un programa deficiente que yo recibí en mi infancia procedente de mi espejo social. \nNo me gusta ese guión inefectivo. Puedo cambiar».  \nEl segu ndo hábito es la creación primera o mental. Se basa en la imaginación (la capacidad para visualizar, \npara ver lo potencial, para crear con nuestras mentes lo que en el presente no podemos ver con nuestros ojos) \ny en la conciencia moral (que es la capacidad  para de tectar nuestra propia singularidad y las directrices \npersonales, morales y éticas que nos permiten llevarla felizmente a cabo). Supone el contacto profundo con', meta

### Una vez ya lo tenemos guardado en disco

In [13]:
chroma_local = Chroma(persist_directory="./vectordb", embedding_function=OpenAIEmbeddings())

In [14]:
retriever = chroma_local.as_retriever()

In [15]:
retriever.get_relevant_documents('afilar la sierra') #consulta que nos va a traer los documentos mas reelevantes teniendo en cuenta nuestra consulta

[Document(page_content='su jardín florido no reserva una parcela para la maleza.  \n \nCuando ya tenemos autoconciencia, debemos elegir los prop ósitos y principios que pondremos en práctica \nen la vida; de otro modo alguna otra cosa llenará el vacío, y perderemos nuestra autoconcien cia para ser como \nanimales rastreros, cuya finalidad primordial es la supervivencia y la reproducción. Las personas que viven en \nese nivel no están viviendo, están siendo vividas. Están reaccionando, no to man conciencia de los dones \nsingulares que dormitan sin desarro llarse en su interior.  \nY para desarrollarlos no hay atajos. Rige la ley de la cosecha: siempre recogem os lo que sembramos, ni \nmás ni menos. La ley de la justicia es inmutable, y cuanto más coherentes seamos con los prin cipios correctos, \nmejor será nuestro juicio sobre cómo funciona el mundo, y más precisos nuestros paradigmas, nuestros mapas', metadata={'page': 186, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
 Docu

# Version gratis

In [2]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis

c:\Users\fcore\RAG basico\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\fcore\RAG basico\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


### Ahora la VectorDB es persistente, se guardan los datos en esta carpeta

In [3]:
import chromadb

persistent_client = chromadb.PersistentClient(path='./vectordb_gratis')

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) #usamos menos tokens que en el anterior debido a que este modelo es inferior
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb_gratis')

retriever = vectorstore.as_retriever()

### Resultados

In [8]:
retriever.get_relevant_documents('afilar la sierra') 

[Document(page_content='—¡Se le ve exhausto! —exclama usted —. ¿Cuánto t iempo hace que trabaja?  \n—Más de cinco horas, y estoy molido. Esto no es sencillo.  \n—¿Por qué no hace una pausa durante unos minutos y afila la sierra? —pregunta usted —. Estoy seguro de \nque cortaría mucho más rápido.  \n—No tengo tiempo para afilar la sierra —dice el hombre enfáti camente —. Estoy demasiado ocupado \naserrando.  \nEl séptimo hábito consiste en tomar tiempo para afilar la sierra. Engloba a todos los otros hábitos del', metadata={'page': 176, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
 Document(page_content='y conoceremos el lugar por primera vez.', metadata={'page': 27, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
 Document(page_content='de unas veinte personas, y una de ellas cayó enferma. Entonces llamó a su esposa, que estaba en Calif ornia,', metadata={'page': 83, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
 Document(page_content='interdependencia  • Logr

### LLM

In [14]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOllama(model='llama3', temperature=0)

### Crear prompt

In [16]:
system_prompt = (
    "Tú eres un asistente para tareas de respuesta a preguntas."
    "Usa los siguientes fragmentos de contexto recuperado para responder "
    "la pregunta. Si no sabes la respuesta, di que no "
    "sabes. Usa un máximo de tres oraciones y mantén la "
    "respuesta concisa."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


chain = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, chain)

### Obtener nuestra respuesta

In [17]:
results = rag.invoke({"input": "que es afilar la sierra?"})

In [18]:
results

{'input': 'que es afilar la sierra?',
 'context': [Document(page_content='y conoceremos el lugar por primera vez.', metadata={'page': 27, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
  Document(page_content='—¡Se le ve exhausto! —exclama usted —. ¿Cuánto t iempo hace que trabaja?  \n—Más de cinco horas, y estoy molido. Esto no es sencillo.  \n—¿Por qué no hace una pausa durante unos minutos y afila la sierra? —pregunta usted —. Estoy seguro de \nque cortaría mucho más rápido.  \n—No tengo tiempo para afilar la sierra —dice el hombre enfáti camente —. Estoy demasiado ocupado \naserrando.  \nEl séptimo hábito consiste en tomar tiempo para afilar la sierra. Engloba a todos los otros hábitos del', metadata={'page': 176, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
  Document(page_content='mientras yo le describo a la joven, y usted sabe que voy a re cibir con el mismo respeto la descripción que me va \na hacer de la an ciana. Ambos n os comprometemos a tratar de compren

In [19]:
results['answer']

'En el contexto del diálogo, "afilar la sierra" se refiere a tomar un descanso o una pausa para rejuvenecer y mejorar el rendimiento. En este caso, el hombre está trabajando más de cinco horas sin parar y su amigo le sugiere que tome un momento para relajarse y volver a enfocarse en su tarea.'

In [21]:
rag.invoke({"input": "cual es el habito 5?"})

{'input': 'cual es el habito 5?',
 'context': [Document(page_content='hábitos 4, 5 y 6. Comprender la secuencia ayuda a controlar el desarrollo con más efectividad, pero no le \nsugiero al lector que se  aísle  durante varios años hasta desarro llar completamente los hábitos 1, 2 y 3.  \nComo parte de un mundo interdependiente, uno tiene que rela cionarse con ese mundo día tras día. Pero \nlos problemas más apre miantes de ese mundo pueden fácilmente oscurecer las causas de carácter crónico. La', metadata={'page': 31, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
  Document(page_content='Los 7 hábitos de la gente altamente efectiva  Stephen R. Covey  \n147 El corazó n tiene sus razones  \nque la razón no com prende.  \n \nPASCAL  \n \nSuponga el lector que tiene problemas en la vista y que decide vi sitar a un oculista. Después de escuchar \nbrevemente su queja, el ocu lista se saca las gafas y se las entrega.  \n—Póngase éstas —le dic e—. Yo he usado este par de gafas du rant

### Una vez ya lo tenemos guardado en disco

In [22]:
chroma_local = Chroma(persist_directory="./vectordb_gratis", embedding_function=embeddings)

In [23]:
retriever = chroma_local.as_retriever()

In [25]:
retriever.get_relevant_documents('habito 6')

[Document(page_content='Los 7 hábitos de la gente altamente efectiva  Stephen R. Covey  \n106 Pero si uno tiene profundamente internalizado el segundo hábito en el corazón y en la mente, serán esos \nvalores los que nos guíen. Se puede subordinar con integridad la agenda a los valores. Se puede adaptar, se \npuede ser flexible. Uno no debe  sentirse culpable cuando no cumple con la programación horaria o cuando hay \nque cambiarla.  \n \nProgresos de la cuarta generación', metadata={'page': 106, 'source': './los7HabitosGenteAltamenteEfectiva.pdf'}),
 Document(page_content='hábitos 4, 5 y 6. Comprender la secuencia ayuda a controlar el desarrollo con más efectividad, pero no le \nsugiero al lector que se  aísle  durante varios años hasta desarro llar completamente los hábitos 1, 2 y 3.  \nComo parte de un mundo interdependiente, uno tiene que rela cionarse con ese mundo día tras día. Pero \nlos problemas más apre miantes de ese mundo pueden fácilmente oscurecer las causas de carácter cr